In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [2]:
def get_idx_without_outliers(data, m=3):
    valid_idx = np.where(abs(data - np.mean(data)) < m * np.std(data))
    return valid_idx

In [3]:
df_city_info = pd.read_csv('../data/df_city_info.csv')
df_city_info.head()

,city_id,lon,lat,UI,EVI,cz,cz_name
0,1,172.615085,-43.531121,0.390941,0.598189,2,Temperate
1,2,174.794893,-36.935893,0.418084,0.609491,2,Temperate
2,3,-4.006245,5.378797,0.383688,0.480865,1,Tropical
3,4,115.771042,33.831816,0.297725,0.564582,2,Temperate
4,5,3.156077,36.698236,0.403523,0.464999,2,Temperate


In [4]:
def get_df_all(year_list=[2002, 2005, 2008, 2011, 2014, 2017]):
    df_all = pd.DataFrame()
    for year in year_list:
        df_one_year = pd.read_csv('../data/df_{}.csv'.format(year))
        df_one_year['year'] = year
        df_all = pd.concat([df_all, df_one_year], axis=0)
    df_all = df_all[df_all['UI'] > 0].reset_index(drop=True)
    df_all = df_all.reset_index(drop=True)
    valid_idx_list = get_idx_without_outliers(df_all['EVI'], m=5)
    df_all = df_all.iloc[valid_idx_list].reset_index(drop=True)
    df_all['EVI'] = preprocessing.minmax_scale(df_all['EVI'])
    df_all = pd.merge(left=df_all, right=df_city_info[['city_id', 'cz', 'cz_name']], on='city_id', how='left')
    return df_all

In [5]:
year_list = [2002, 2005, 2008, 2011, 2014, 2017]
df_all = get_df_all()

In [6]:
# df_all.to_csv('../data/df_all.csv', index=False)
df_all.head()

,lon,lat,EVI,NDVI,NDVI_nbar,NIRv,UI,crop,snow,tree,water,city_id,year,cz,cz_name
0,172.703359,-43.610961,0.385504,0.455472,0.214569,0.020042,0.0828,0.00,0.0,0.04,0.40,1,2002,2,Temperate
1,172.725817,-43.610961,0.234333,0.211667,-0.218050,-0.009084,0.0132,0.00,0.0,0.00,0.85,1,2002,2,Temperate
2,172.694376,-43.606470,0.496266,0.608582,0.505586,0.077739,0.0660,0.00,0.0,0.37,0.15,1,2002,2,Temperate
3,172.698868,-43.606470,0.522371,0.669623,0.611649,0.105050,0.0768,0.00,0.0,0.44,0.00,1,2002,2,Temperate
4,172.703359,-43.606470,0.522371,0.669623,0.611649,0.105050,0.1416,0.01,0.0,0.29,0.00,1,2002,2,Temperate
